# [LRU Cache](https://leetcode.com/problems/lru-cache/description/)

Design a data structure to store key-value pairs with fast access and update. When capacity is exceeded, evict the least recently used item.
## Strategy
> Without capacity, this would be a simple hash, so you'll need another data structure. The O(1) requirement hints that additional structure must also have O(1) ability, like a linked list.
- Use a [doubly linked list](../resources/linked-list.ipynb) to keep track of usage order (most recent at head).
- Use a hash map for O(1) access to nodes.
- On get, move accessed node to head.
- On put, insert new node at head, remove tail if over capacity.

## Remember
- When removing a doubling linked list element with dummy nodes, you can use a generic code to remove any element


```c++
/**
 * Node structure for doubly-linked list
 * Each node stores a key-value pair and maintains pointers to adjacent nodes
 */
struct Node {
    int key;
    int val;
    Node* next{nullptr};
    Node* prev{nullptr};
    
    // Constructor initializes key and value, pointers default to nullptr
    Node(int k, int v): key(k), val(v) {}
};

/**
 * LRU (Least Recently Used) Cache Implementation
 * Uses HashMap + Doubly-Linked List for O(1) get and put operations
 * 
 * Data Structure Design:
 * - HashMap: Provides O(1) key lookup to find nodes
 * - Doubly-Linked List: Maintains insertion order and allows O(1) removal/addition
 * - Dummy head/tail: Simplifies edge case handling (no null checks needed)
 */
class LRUCache {
public:
    int capacity;                              // Maximum number of key-value pairs
    unordered_map<int, Node*> cache;          // Hash map for O(1) key -> node lookup
    Node* head{new Node(-1, -1)};             // Dummy head (most recently used end)
    Node* tail{new Node(-1, -1)};             // Dummy tail (least recently used end)
    
    /**
     * Constructor: Initialize LRU Cache with given capacity
     * Sets up the doubly-linked list with dummy head and tail connected
     */
    LRUCache(int capacity): capacity(capacity) {
        // Initialize empty doubly-linked list: head ←→ tail
        head->next = tail;
        tail->prev = head;
    }
    
    /**
     * Destructor: Clean up all allocated memory to prevent memory leaks
     * Must delete all nodes in the list plus the dummy head/tail
     */
    ~LRUCache() {
        // Delete all data nodes between head and tail
        while (head->next != tail) {
            Node* nodeToDelete = head->next;
            remove(nodeToDelete);
            delete nodeToDelete;
        }
        // Delete dummy nodes
        delete head;
        delete tail;
    }
    
    /**
     * Get value by key
     * If key exists: move to front (mark as recently used) and return value
     * If key doesn't exist: return -1
     * Time Complexity: O(1)
     */
    int get(int key) {
        // Check if key exists in hash map
        if (!cache.contains(key)) {
            return -1;  // Key not found
        }
        
        // Key found: move node to front to mark as recently used
        Node *node = cache[key];
        remove(node);   // Remove from current position
        add(node);      // Add to front (after head)
        return node->val;
    }
    
    /**
     * Insert or update key-value pair
     * If key exists: update value and move to front
     * If key doesn't exist: create new node, add to front, handle capacity
     * Time Complexity: O(1)
     */
    void put(int key, int value) {
        // Case 1: Key already exists - update and move to front
        if (cache.contains(key)) {
            Node *oldNode = cache[key];
            remove(oldNode);        // Remove from list
            delete oldNode;         // Free memory
            // Note: We'll create a new node below rather than updating in-place
        }
        
        // Case 2: Create new node and add to front
        Node *node = new Node(key, value);
        cache[key] = node;      // Add to hash map
        add(node);              // Add to front of list
        
        // Handle capacity constraint: remove LRU item if over capacity
        if (cache.size() > capacity) {
            // LRU item is always right before tail (tail->prev)
            Node *nodeToDelete = tail->prev;
            remove(nodeToDelete);                   // Remove from list
            cache.erase(nodeToDelete->key);         // Remove from hash map
            delete nodeToDelete;                    // Free memory
        }
    }
    
private:
    /**
     * Add node right after head (mark as most recently used)
     * This is where we insert new nodes and move accessed nodes
     * List structure: head ←→ newNode ←→ oldFirstNode ←→ ... ←→ tail
     */
    void add(Node *node) {
        Node *previousEnd = head->next;     // Current first data node
        
        // Insert node between head and previousEnd
        head->next = node;                  // head points to new node
        node->prev = head;                  // new node points back to head
        node->next = previousEnd;           // new node points to old first
        previousEnd->prev = node;           // old first points back to new node
    }
    
    /**
     * Remove node from its current position in the list
     * Works for any node in the middle of the list (not head/tail)
     * Updates the previous and next nodes to skip over the removed node
     */
    void remove(Node *node) {
        // Bridge the gap: make prev and next nodes point to each other
        node->prev->next = node->next;      // Previous node skips over current
        node->next->prev = node->prev;      // Next node points back to previous
        // Note: We don't delete the node here, just unlink it
    }
};
```

```c++

## Solution



In [ ]:
class Node {
  constructor(key?: number, val?: number) {
    this.key = key ?? null
    this.val = val ?? null
    this.next = null
    this.prev = null
  }
}

class LRUCache {
  constructor(capacity: number) {
      this.map = new Map<number, Node>()
      this.capacity = capacity
      this.head = new Node()
      this.tail = new Node()
      this.head.next = this.tail
      this.tail.prev = this.head
  }

  moveToFront(key: number) {
      const node: Node = this.map.get(key)

      // head node can remain where it is
      if(this.head.next !== node) {
        // remove node
        node.prev.next = node.next
        node.next.prev = node.prev
        
        node.next = this.head.next
        node.prev = this.head
        this.head.next.prev = node
        this.head.next = node
      }
  }

  get(key: number): number {
    if (this.map.has(key)) {
        this.moveToFront(key)
        return this.map.get(key).val
    }

    return -1
  }

  put(key: number, value: number): void {
    // just update if it already exists
    if (this.map.has(key)) {
      this.map.get(key).val = value
      this.moveToFront(key)
    } else {
      // remove the LRU
      if ((this.map.size + 1) > this.capacity) {
        // cull
        const lru: Node = this.tail.prev
        lru.prev.next = this.tail
        this.tail.prev = lru.prev

        // remove from the map
        this.map.delete(lru.key)
      }

      // store in front
      const node: Node = new Node(key, value)
      node.prev = this.head
      node.next = this.head.next
      this.head.next.prev = node
      this.head.next = node

      // store in map
      this.map.set(key, node)
    }
  }
}

## Test Cases



In [48]:
import { assertEquals } from "https://deno.land/std/testing/asserts.ts";

Deno.test("LRU Cache: put key 1", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  assertEquals(cache.get(1), 1);
});

Deno.test("LRU Cache: put key 2", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(2, 2);
  assertEquals(cache.get(2), 2);
});

Deno.test("LRU Cache: evict least recently used key", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(2, 2);
  cache.get(1);          // Access key 1
  cache.put(3, 3);       // Evicts key 2
  assertEquals(cache.get(2), -1);
});

Deno.test("LRU Cache: update value of existing key", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(1, 10);
  assertEquals(cache.get(1), 10);
});

Deno.test("LRU Cache: evict after capacity exceeded", () => {
  const cache = new LRUCache(2);
  cache.put(1, 1);
  cache.put(2, 2);
  cache.put(3, 3); // Evicts key 1
  assertEquals(cache.get(1), -1);
});

LRU Cache: put key 1 ... ok (1ms)
LRU Cache: put key 2 ... ok (0ms)
LRU Cache: evict least recently used key ... ok (0ms)
LRU Cache: update value of existing key ... ok (0ms)
LRU Cache: evict after capacity exceeded ... ok (0ms)

ok | 5 passed | 0 failed (2ms)
